In [1]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

import csv, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

pd.set_option('display.max_columns', 500)
curr_dir = '/home/souche/qiongjiu/hgc/'

In [2]:
data = pd.read_csv(curr_dir+"2020-11-02版带count处理完成的零售数据m.csv", header = 0, low_memory=False)
data.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1701843 entries, 0 to 1701842
Data columns (total 61 columns):
car_id                      1701843 non-null int64
brand_code                  1701606 non-null object
brand_name                  1701606 non-null object
series_code                 1701606 non-null object
series_name                 1701606 non-null object
model_code                  1701843 non-null object
model_name                  1701606 non-null object
mileage_std                 1701843 non-null float64
color_name                  1701766 non-null object
first_license_plate_date    1701843 non-null object
province_name               1701843 non-null object
city_name                   1701843 non-null object
price                       1701843 non-null float64
pay_time                    1701843 non-null object
license_month               1701843 non-null int64
residual                    1701843 non-null float64
days                        1701843 non-null int64
lic

In [7]:
car = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header = 0, low_memory=False)
car=car.drop(['model_year','new_car_price', 'driving_mode','gear_box_type','country_id',
 'import_type','intake_type', 'fuel_form','quality_mile', 'quality_year',
 'year_1','year_2','year_3','year_4','year_5','year_6','year_7','year_8',
 'year_9', 'year_10','year_11','year_12','year_13','year_14','year_15',
 'year_16', 'car_body','rate','rate_count'],axis=1)

#print(car.shape)

In [8]:
data = pd.merge(data, car, how="inner", on="model_code")
vec = pd.read_csv(curr_dir+"2020-11-02版-TF-IDF权重的词向量.csv", dtype={'model_code': str}, header = 0)
vec = vec.drop(['model_name'], axis=1)

In [9]:
data = pd.merge(data, vec, how="inner", on="model_code")
#print(data.shape)

In [10]:
data_A = data[(data['level']=="缺失") | 
              (data['level']=="A00") | 
              (data['level']=="A0") | 
              (data['level']=="A")]
print("筛选出A级以下的车数据量为:%d" % data_A.shape[0])
print("筛选出A级以下的车的model_code为:%d" % len(data_A['model_code'].unique()))
print("包含的车型级别:", data_A['level'].unique())

筛选出A级以下的车数据量为:871149
筛选出A级以下的车的model_code为:18371
包含的车型级别: ['A0' 'A' 'A00' '缺失']


In [12]:
data_X = data_A.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8'], axis=1)
data_y = data_A['residual']

print(data_X.shape, len(data_y))


(871149, 282) 871149


In [13]:
for col in ['ypred', 'err']:
    if col in data_X.columns:
        del data_X[col]

In [15]:
import lightgbm as lgb
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression',
                        learning_rate = 0.03, 
                        n_estimators = 3000,
                        max_depth = 9,
                        num_leaves = 200, 
                        subsample = 1, 
                        colsample_bytree = 0.3, 
                        min_child_samples = 50, 
                        n_jobs = 8,
                        verbose = 1)

gbm.fit(data_X, 
        data_y, 
        verbose = True)



LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
       learning_rate=0.03, max_depth=9, min_child_samples=50,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=3000,
       n_jobs=8, num_leaves=200, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [16]:
fearture_columns = list(data_X.columns)

In [17]:
data_A['ypred'] = gbm.predict(data_A[fearture_columns])
data_A['err'] = abs(data_A['ypred'] - data_A['residual']) / data_A['residual']
data_A[data_A['err']>=0.1].shape


(204295, 313)

In [19]:
def data3m_pinggu(data_df,col):
    total = data_df.shape[0]
    num_3 = data_df.loc[(data_df[col]>=-0.03 ) & (data_df[col]<=0.03 )].shape[0]
    print("P<3%: ",round(num_3/total,4))
    num_5 = data_df.loc[(data_df[col]>=-0.05 ) & (data_df[col]<=0.05 )].shape[0]
    print("P<5%: ",round(num_5/total,4))
    num_8 = data_df.loc[(data_df[col]>=-0.08 ) & (data_df[col]<=0.08 )].shape[0]
    print("P<8%: ",round(num_8/total,4))
    num_10 = data_df.loc[(data_df[col]>=-0.1 ) & (data_df[col]<=0.1 )].shape[0]
    print("P<10%: ",round(num_10/total,4))
    num_20 = data_df.loc[(data_df[col]>=-0.2 ) & (data_df[col]<=0.2 )].shape[0]
    print("P<20%: ",round(num_20/total,4))
data3m_pinggu(data_A.loc[data_A['pay_time'] > '2020-08-01'],'err')


P<3%:  0.3468
P<5%:  0.5233
P<8%:  0.7013
P<10%:  0.7762
P<20%:  0.9313


In [25]:
data_A = data_A[data_A['err']<0.1]
#打乱数据
data_A = shuffle(data_A)

data_X = data_A.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8','count', 'ypred', 'err'], axis=1)
data_y = data_A['residual']
print(data_X.shape, len(data_y))


(666854, 282) 666854


In [27]:
gbm

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
       learning_rate=0.03, max_depth=9, min_child_samples=50,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=3000,
       n_jobs=8, num_leaves=200, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [28]:
! pip install sklearn2pmml

You should consider upgrading via the '/home/souche/projects/calculation/py3dev_new/bin/python3 -m pip install --upgrade pip' command.


In [29]:
for i in range(15):
    data_X = data_X.rename(columns={"country_id"+str(i):"Country_Id"+str(i)})


In [31]:
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
import lightgbm as lgb

pipeline = PMMLPipeline([("regressor", lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression',
                        learning_rate = 0.03, 
                        n_estimators = 3000,
                        max_depth = 9,
                        num_leaves = 200, 
                        subsample = 1, 
                        colsample_bytree = 0.3, 
                        min_child_samples = 50, 
                        n_jobs = 8,
                        verbose = 1))])
pipeline.fit(data_X, data_y)

ph = curr_dir+"enterprise2_AAA_level_dfc_retail.pmml"
sklearn2pmml(pipeline, ph, with_repr = True)


In [30]:
joblib.dump(pipeline,curr_dir+'A_level_dfc_retail_model_test_2020-11-02_')


['/home/souche/qiongjiu/hgc/A_level_dfc_retail_model_test_2020-11-02_']

In [31]:
# 上传到 oss上 
ph = curr_dir+"enterprise2_AAA_level_dfc_retail.pmml"
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/2020-11-02_/enterprise2_AAA_level_dfc_retail.pmml", ph)


DAYU_HOME : /home/souche/projects/datacenter-etl-v2
[2020-11-11 14:58:59,265] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2020-11-11 14:58:59,292] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2020-11-11 14:59:01,624] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/2020-11-02_/enterprise2_AAA_level_dfc_retail.pmml .


In [32]:
# 上传到 oss上 
ph = curr_dir+"A_level_dfc_retail_model_test_2020-11-02_"
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/2020-11-02_/A_level_dfc_retail_model_test_2020-11-02", ph)


[2020-11-11 14:59:03,104] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/2020-11-02_/A_level_dfc_retail_model_test_2020-11-02 .
